# Installing and Importing Libraries

In [ ]:
from keras.models import load_model
from collections import deque
import numpy as np
import argparse
import pickle
import cv2
import telepot
from datetime import datetime
import pytz
from PIL import Image,ImageEnhance
import pyrebase
import os
from matplotlib import pyplot as plt
from matplotlib.patches import Rectangle, Circle
from mtcnn.mtcnn import MTCNN

# Creating Firebase Setup

Package                      Version
---------------------------- ------------
absl-py                      2.0.0
accelerate                   0.26.1
aiohttp                      3.9.1
aiosignal                    1.3.1
altair                       5.2.0
anyio                        3.7.1
argon2-cffi                  23.1.0
argon2-cffi-bindings         21.2.0
arrow                        1.2.3
asgiref                      3.7.2
asttokens                    2.2.1
astunparse                   1.6.3
async-lru                    2.0.4
attrs                        23.1.0
audioread                    3.0.1
Babel                        2.12.1
backcall                     0.2.0
beautifulsoup4               4.12.2
bleach                       6.0.0
blinker                      1.7.0
boto3                        1.34.14
botocore                     1.34.14
Brotli                       1.1.0
cachetools                   5.3.2
certifi                      2023.11.17
cffi                         1.